# Camera Calibration

Renato Sérgio Lopes Júnior

In [1]:
import numpy as np
import cv2

## Creating matrix A and computing the SVD

In [47]:
N = 3 # amount of points
image_coords = np.array([[0, 1], [2, 1], [3, 3]])
world_coords = np.array([[10, 15, 1], [21, 12, 3], [32, 30, 10]])

def compute_SVD(A):
    U, S, Vt = np.linalg.svd(A, full_matrices=False)
    return U, S, Vt

In [57]:
def create_matrix_A(image_coords, world_coords, N):
    A = np.zeros([N, 8])
    for i in range(N):
        x = image_coords[i][0]
        y = image_coords[i][1]
        X = world_coords[i][0]
        Y = world_coords[i][1]
        Z = world_coords[i][2]
        A[i][0] = x*X
        A[i][1] = x*Y
        A[i][2] = x*Z
        A[i][3] = x
        A[i][4] = -y*X
        A[i][5] = -y*Y
        A[i][6] = -y*Z
        A[i][7] = -y
    return A

# Create matrix A
A = create_matrix_A(image_coords, world_coords, N)
# Compute SVD of A
U, S, Vt = compute_SVD(A)
# Get V
index = np.argmin(S) # Get index of lowest element
V = Vt[index]

## Computing $R$, $T$ and $f_x$

In [50]:
# Compute gamma and alpha
gamma = np.sqrt((V[0]**2) + (V[1]**2) + (V[2]**2))
alpha = np.sqrt((V[4]**2) + (V[5]**2) + (V[6]**2))/gamma

In [45]:
# Recover the first two rows of R
R = np.zeros([3, 3])
R[1][0] = V[0]/gamma
R[1][1] = V[1]/gamma
R[1][2] = V[2]/gamma
R[0][0] = V[4]/(gamma*alpha)
R[0][1] = V[5]/(gamma*alpha)
R[0][2] = V[6]/(gamma*alpha)

# Compute the third row of R
R[2] = np.cross(R[0], R[1])

# Enforce orthogonality in R
U, S, Vt = compute_SVD(R)
R = np.matmul(U, Vt.transpose())np.matmul(R, R.transpose())

In [46]:
# Recover the first two components of T
T = np.zeros([3])
T[1] = V[3]/gamma
T[0] = V[7]/(gamma*alpha)

In [56]:
# Check if sign must change
P = 0 # Selected point
if image_coords[P][0]*(R[0][0]*world_coords[P][0] + R[0][1]*world_coords[P][1] + R[0][2]*world_coords[P][2] + T[0]) > 0:
    R[0] = (-1) * R[0]
    R[1] = (-1) * R[1]
    T[0] = (-1) * T[0]
    T[1] = (-1) * T[1]

In [62]:
# Estimate Tz and fx
def create_system(image_coords, world_coords, N, R, T):
    A = np.zeros([N, 2])
    b = np.zeros([N])
    for i in range(N):
        A[i][0] = image_coords[i][0]
        A[i][1] = R[0][0]*world_coords[i][0] + R[0][1]*world_coords[i][1] + R[0][2]*world_coords[i][2] + T[0]
        b[i] = -image_coords[i][0]*(R[2][0]*world_coords[i][0] + R[2][1]*world_coords[i][1] + R[2][2]*world_coords[i][2])
    return A, b    

A2, b = create_system(image_coords, world_coords, N, R, T)
S = np.linalg.lstsq(A2, b)[0]
# S = np.matmul(np.matmul(np.linalg.inv(np.matmul(A2.transpose(), A2)), A2.transpose()), b)
T[2] = S[0]
fx = S[1]

/home/renato/anaconda3/envs/computer-vision/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if sys.path[0] == '':


In [63]:
print('R', R)
print('T', T)
print('alpha', alpha)
print('fx', fx)

R [[-0.21822582 -0.95123459 -0.21801434]
 [ 0.10390752 -0.24477682  0.96399561]
 [-0.97035082  0.1877154   0.15225705]]
T [ 0.19136671 -0.06699347 19.41997946]
alpha 0.687411790002347
fx -0.22885944922518464
